# insert into mysql database

In [1]:
import sqlite3

conn=sqlite3.connect('airlines.db')
cursor=conn.cursor()

In [2]:
# Drop and create tables
commands = [
    """DROP TABLE IF EXISTS Aircrafts""",
    """CREATE TABLE Aircrafts(
        AcNumber TEXT NOT NULL UNIQUE,
        Capacity INTEGER NOT NULL,
        MfdBy TEXT NOT NULL,
        MfdOn DATETIME NOT NULL
    )""",
    """DROP TABLE IF EXISTS Route""",
    """CREATE TABLE Route(
        RtID INTEGER PRIMARY KEY,
        Airport TEXT NOT NULL,
        Destination TEXT NOT NULL,
        RouteCode TEXT NOT NULL UNIQUE
    )""",
    """DROP TABLE IF EXISTS AirFare""",
    """CREATE TABLE AirFare(
        AfID INTEGER PRIMARY KEY,
        Route INTEGER,
        Fare REAL,
        FSC REAL,
        FOREIGN KEY (Route) REFERENCES Route(RtID)
    )""",
    """DROP TABLE IF EXISTS Flight_Schedule""",
    """CREATE TABLE Flight_Schedule(
        FlID INTEGER PRIMARY KEY,
        FlightDate DATETIME,
        Departure DATETIME,
        Arrival DATETIME,
        Aircraft INTEGER,
        NetFare INTEGER,
        FOREIGN KEY (Aircraft) REFERENCES Aircrafts(AcNumber),
        FOREIGN KEY (NetFare) REFERENCES AirFare(AfID)
    )""",
    """DROP TABLE IF EXISTS Discounts""",
    """CREATE TABLE Discounts(
        DiID INTEGER PRIMARY KEY,
        Title TEXT,
        Amount INTEGER,
        Description TEXT
    )""",
    """DROP TABLE IF EXISTS Charges""",
    """CREATE TABLE Charges(
        ChID INTEGER PRIMARY KEY,
        Title TEXT,
        Amount INTEGER,
        Description TEXT
    )""",
    """DROP TABLE IF EXISTS Countries""",
    """CREATE TABLE Countries(
        CtID INTEGER PRIMARY KEY,
        CountryName TEXT NOT NULL
    )""",
    """DROP TABLE IF EXISTS State""",
    """CREATE TABLE State(
        StID INTEGER PRIMARY KEY,
        StateName TEXT,
        Country INTEGER,
        FOREIGN KEY (Country) REFERENCES Countries(CtID)
    )""",
    """DROP TABLE IF EXISTS Contact_Details""",
    """CREATE TABLE Contact_Details(
        CnID INTEGER PRIMARY KEY,
        Email TEXT NOT NULL,
        Cell TEXT NOT NULL,
        Tel TEXT,
        Street TEXT,
        State INTEGER NOT NULL,
        FOREIGN KEY (State) REFERENCES State(StID)
    )""",
    """DROP TABLE IF EXISTS Passengers""",
    """CREATE TABLE Passengers(
        PsID INTEGER PRIMARY KEY,
        Name TEXT NOT NULL,
        Address TEXT NOT NULL,
        Age INTEGER NOT NULL,
        Nationality TEXT NOT NULL,
        Contacts INTEGER NOT NULL,
        FOREIGN KEY (Contacts) REFERENCES Contact_Details(CnID)
    )""",
    """DROP TABLE IF EXISTS Branches""",
    """CREATE TABLE Branches(
        BrID INTEGER PRIMARY KEY,
        Center TEXT NOT NULL,
        Address TEXT NOT NULL,
        State INTEGER,
        FOREIGN KEY (State) REFERENCES State(StID)
    )""",
    """DROP TABLE IF EXISTS Employee""",
    """CREATE TABLE Employee(
        EmpID INTEGER PRIMARY KEY,
        Name TEXT NOT NULL,
        Address TEXT NOT NULL,
        Branch INTEGER NOT NULL,
        Designation TEXT NOT NULL,
        Email TEXT NOT NULL,
        Tel TEXT NOT NULL,
        Ext INTEGER,
        FOREIGN KEY (Branch) REFERENCES Branches(BrID)
    )""",
    """DROP TABLE IF EXISTS Transactions""",
    """CREATE TABLE Transactions(
        TsID INTEGER PRIMARY KEY,
        BookingDate DATETIME,
        DepartureDate DATETIME,
        Passenger INTEGER,
        Flight INTEGER,
        Type INTEGER,
        Employee INTEGER,
        Charges INTEGER,
        Discount INTEGER,
        FOREIGN KEY (Passenger) REFERENCES Passengers(PsID),
        FOREIGN KEY (Flight) REFERENCES Flight_Schedule(FlID),
        FOREIGN KEY (Employee) REFERENCES Employee(EmpID),
        FOREIGN KEY (Charges) REFERENCES Charges(ChID),
        FOREIGN KEY (Discount) REFERENCES Discounts(DiID)
    )""",
    
]

# Execute each command
for command in commands:
    cursor.execute(command)

# Commit changes and close connection
conn.commit()
#conn.close()

print("SQLite database schema created successfully!")

SQLite database schema created successfully!


In [3]:
# connect to mysql database 

# config={
#     'user': 'root',
#     'password': 'Helloworld@123',
#     'host': 'localhost',
#     'port': 3305,  # Update the port number to 3305 because in installation i gave port 3305
#     'database':'airlines'
#}

# Insert Mass Data

In [4]:
# insert records into the database 
import pandas as pd
import sqlite3 as sql
import os
%config SqlMagic.style = '_DEPRECATED_DEFAULT'


In [5]:
cnn=sql.connect('airlines.db')

In [6]:
path=os.path.abspath(os.path.join(os.getcwd(),".."))
path=os.path.join(path,'data')
path

'd:\\pythonProjects\\airlines_management\\data'

In [7]:
# load the csvs to the tables
filepaths = {
    "Aircrafts": os.path.join(path, "Aircrafts.csv"),
    "AirFare": os.path.join(path, "AirFare.csv"),
    "Branches": os.path.join(path, "Branches.csv"),
    "Charges": os.path.join(path, "Charges.csv"),
    "Contact_Details": os.path.join(path, "Contact_Details.csv"),
    "Countries": os.path.join(path, "Countries.csv"),
    "Discounts": os.path.join(path, "Discounts.csv"),
    "Employee": os.path.join(path, "Employee.csv"),
    "Flight_Schedule": os.path.join(path, "Flight_Schedule.csv"),
    "Passengers": os.path.join(path, "Passengers.csv"),
    "Route": os.path.join(path, "Route.csv"),
    "State": os.path.join(path, "State.csv"),
    "Transactions": os.path.join(path, "Transactions.csv"),
}

In [8]:
for table_name, filepath in filepaths.items():
    if os.path.exists(filepath):  # Check if the file exists
        # Read CSV into DataFrame
        df = pd.read_csv(filepath)
        
        # Load DataFrame into SQLite
        df.to_sql(table_name, cnn, if_exists="replace", index=False)
        print(f"Loaded {table_name} into SQLite database.")
    else:
        print(f"File {filepath} does not exist. Skipping {table_name}.")

# Close the database connection
#cnn.close()

print("All tables have been loaded into the SQLite database.")

Loaded Aircrafts into SQLite database.
Loaded AirFare into SQLite database.
Loaded Branches into SQLite database.
Loaded Charges into SQLite database.
Loaded Contact_Details into SQLite database.
Loaded Countries into SQLite database.
Loaded Discounts into SQLite database.
Loaded Employee into SQLite database.
Loaded Flight_Schedule into SQLite database.
Loaded Passengers into SQLite database.
Loaded Route into SQLite database.
Loaded State into SQLite database.
Loaded Transactions into SQLite database.
All tables have been loaded into the SQLite database.


# create triggers

In [9]:
# triggers 
commands=[
    """Drop TABLE IF EXISTS LOGS""",
    """CREATE TABLE LOGS(
        LogID Integer PRIMARY KEY AUTOINCREMENT,
        OPERATION TEXT,
        LogTimeStamp DATETIME DEFAULT CURRENT_TIMESTAMP,
        DATA TEXT
    );
    """,
""" DROP TRIGGER IF EXISTS InsertAircraftLog;""",
    """
CREATE TRIGGER InsertAircraftLog
AFTER INSERT ON Aircrafts
FOR EACH ROW
BEGIN
    INSERT INTO LOGS (OPERATION, DATA)
    VALUES (
        'INSERT',
         'Aircraft: ' || NEW.AcNumber || ', ' || NEW.Capacity || ', ' || NEW.MfdBy || ', ' || NEW.MfdOn
        );
END;
""",
""" DROP TRIGGER IF EXISTS InsertScheduleLog;""",
    """
CREATE TRIGGER InsertScheduleLog
AFTER INSERT ON Flight_Schedule
FOR EACH ROW
BEGIN
    INSERT INTO LOGS (OPERATION, DATA)
    VALUES (
        'INSERT',
          'Flight_Schedule: ' || NEW.FlID || ', ' || NEW.FlightDate || ', ' || NEW.Departure || ', ' || 
        NEW.Arrival || ', ' || NEW.Aircraft || ', ' || NEW.NetFare
        );
END;
""",
""" 
CREATE TRIGGER InsertPassengerLog
AFTER INSERT ON Passengers
FOR EACH ROW
BEGIN
    INSERT INTO LOGS (OPERATION, DATA)
    VALUES (
        'INSERT',
          'Passengers: ' || NEW.PsID || ', ' || NEW.Name || ', ' || NEW.Address || ', ' || 
        NEW.Age || ', ' || NEW.Nationality || ', ' || NEW.Contacts
        );
END;

""",
""" 
CREATE TRIGGER updatePassengerLog
AFTER UPDATE ON Passengers
FOR EACH ROW
BEGIN
    INSERT INTO LOGS (OPERATION, DATA)
    VALUES (
        'UPDATE',
        'Passengers: ' || 'Old PsID: ' || OLD.PsID || ', New PsID: ' || NEW.PsID || ', ' ||
        'Old Name: ' || OLD.Name || ', New Name: ' || NEW.Name || ', ' ||
        'Old Address: ' || OLD.Address || ', New Address: ' || NEW.Address || ', ' ||
        'Old Age: ' || OLD.Age || ', New Age: ' || NEW.Age || ', ' ||
        'Old Nationality: ' || OLD.Nationality || ', New Nationality: ' || NEW.Nationality || ', ' ||
        'Old Contacts: ' || OLD.Contacts || ', New Contacts: ' || NEW.Contacts
    );
END;

""",
""" 
CREATE TRIGGER IF NOT EXISTS deletePassengerLog
AFTER DELETE ON Passengers
FOR EACH ROW
BEGIN
    INSERT INTO LOGS (OPERATION, DATA)
    VALUES (
        'DELETE',
        'Passengers: ' || OLD.PsID || ', Name: ' || OLD.Name || ', Address: ' || OLD.Address || ', ' ||
        'Age: ' || OLD.Age || ', Nationality: ' || OLD.Nationality || ', Contacts: ' || OLD.Contacts
    );
END;

""",
""" 
CREATE TRIGGER IF NOT EXISTS insertTransactionLog
AFTER INSERT ON Transactions
FOR EACH ROW
BEGIN
    INSERT INTO LOGS (OPERATION, DATA)
    VALUES (
        'INSERT',
        'Booking: '|| NEW.TsID || ', ' || NEW.BookingDate || ', ' || NEW.DepartureDate || ', ' || 
        NEW.Passenger || ', ' || NEW.Flight || ', ' || NEW.Type || ', ' || 
        NEW.Employee || ', ' || NEW.Charges || ', ' || NEW.Discount
    );
END;

""",

""" 
CREATE TRIGGER IF NOT EXISTS updateTransactionLog
AFTER UPDATE ON Transactions
FOR EACH ROW
BEGIN
    INSERT INTO LOGS (OPERATION, DATA)
    VALUES (
        'UPDATE',
        'Booking: ' || 'Old: ' || OLD.TsID || ', ' || OLD.BookingDate || ', ' || OLD.DepartureDate || ', ' || 
        OLD.Passenger || ', ' || OLD.Flight || ', ' || OLD.Type || ', ' || 
        OLD.Employee || ', ' || OLD.Charges || ', ' || OLD.Discount || ' | ' ||
        'New: ' || NEW.TsID || ', ' || NEW.BookingDate || ', ' || NEW.DepartureDate || ', ' || 
        NEW.Passenger || ', ' || NEW.Flight || ', ' || NEW.Type || ', ' || 
        NEW.Employee || ', ' || NEW.Charges || ', ' || NEW.Discount
    );
END;
"""


]

# Execute each command
for command in commands:
    cursor.execute(command)

# Commit changes and close connection
conn.commit()
#conn.close()

print("SQLite Triggers created successfully!")

SQLite Triggers created successfully!
